# Preparation

In [63]:
# Use this to install libraries if you find them missing on your system:
#!pip install bs4
#!pip install sklearn
#!pip install nltk
#!pip install gensim
#!pip install lxml
#!pip install keras

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.callbacks import EarlyStopping
import gensim
import gensim.downloader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

In [3]:
EMBEDDING_DIM = 300
MAX_VOCAB_SIZE = 262144
MAX_SEQUENCE_LENGTH = 200

In [4]:
# Adapted from Yoon Kim model
# Kim, Y. (2014). Convolutional neural networks for sentence classification. https://doi.org/10.48550/arXiv.1408.5882

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):

    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Concatenate(axis=-1)(convs)

    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)
    else:
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

# Part 1

In [5]:
df_fake_1 = pd.DataFrame(pd.read_pickle("fake_news_data/clean/fake.pkl"))
df_real_1 = pd.DataFrame(pd.read_pickle("fake_news_data/clean/real.pkl"))

In [6]:
df_fake_1.columns = ["text"]
df_real_1.columns = ["text"]

In [45]:
df_real_1["label"] = True
df_fake_1["label"] = False

In [47]:
df_1 = pd.concat([df_real_1,df_fake_1])
df_1 = df_1.sample(frac = 1)
df_1


,text,label
8953,Airline unions have called for an end to the o...,True
5408,at Low cholesterol and its consequence we have...,False
11392,He finds their reluctance to speak to this mat...,False
9361,S Given the strength of new experts said it wa...,False
7248,originally It looks like everyone is releasing...,False
...,...,...
8715,Knew position long ago,False
11144,had many vocal celebrities on her side during ...,False
5125,whose world chances arrived like buses but end...,True
6,Mandatory vaccinations are about to open up a ...,False


In [9]:
X1 = df_1['text']
y1 = df_1['label']
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3)
X1_train, X1_test, y1_train, y1_test = X_train, X_test, y_train, y_test

In [10]:
tokenizer1 = Tokenizer(num_words=MAX_VOCAB_SIZE,
                      lower=True,
                      char_level=False,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      split=' ')

tokenizer1.fit_on_texts(X1_train.tolist())

train_word_index_1 = tokenizer1.word_index
train_embedding_weights_1 = np.zeros((len(train_word_index_1)+1, EMBEDDING_DIM))

for word, index in train_word_index_1.items():
    if word in word2vec:
        train_embedding_weights_1[index,:] = word2vec[word]
    else:
        train_embedding_weights_1[index,:] = np.random.rand(EMBEDDING_DIM)

print("embedding dim:", train_embedding_weights_1.shape)

embedding dim: (52007, 300)


In [11]:
training_sequences_1 = tokenizer1.texts_to_sequences(X1_train.tolist())
train_cnn_data_1 = pad_sequences(training_sequences_1, maxlen=MAX_SEQUENCE_LENGTH-1)

test_sequences_1 = tokenizer1.texts_to_sequences(X1_test.tolist())
test_cnn_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH-1)

In [12]:
batch_size_1 = 256
labels_1 = 1
num_epochs_1 = 10

In [13]:
model1 = ConvNet(train_embedding_weights_1, MAX_SEQUENCE_LENGTH-1, len(train_word_index_1)+1, EMBEDDING_DIM, labels_1, False)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 199)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 199, 300)             1560210   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 conv1d (Conv1D)             (None, 197, 128)             115328    ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 196, 128)             153728    ['embedding[0][0]']       

In [14]:
tf.config.run_functions_eagerly(True)

In [ ]:
hist = model1.fit(train_cnn_data_1, y1_train, epochs=num_epochs_1, validation_split=0.1, shuffle=True, batch_size=batch_size_1)

In [16]:
y1_predicted = model1.predict(test_cnn_data_1, batch_size=1024, verbose=1)

7/7 [==============================] - 9s 1s/step


In [17]:
y1_pred = np.round(y1_predicted)


In [18]:
accuracy_score(y1_test, y1_pred)

0.8737304835531302

# Part 2

In [25]:
df_fake_2 = pd.DataFrame(pd.read_csv("fake.csv"))
df_fake_2 = df_fake_2.drop(["title", "uuid", "ord_in_thread", "author", "published", "language", "crawled", "site_url",	"country"	,"domain_rank"	,"thread_title",	"spam_score",	"main_img_url",	"replies_count", "participants_count", "likes", "comments", "shares"], axis = 1)
df_fake_2["text"] = df_fake_2["text"].astype(str)

In [26]:
df_fake_2 = df_fake_2.join(pd.get_dummies(df_fake_2['type']))
df_fake_2 = df_fake_2.drop('type', axis = 1)

In [62]:
df_fake_bs = df_fake_2[df_fake_2.bs == True]
df_fake_notbs = df_fake_2[df_fake_2.bs != True]
print("Number of rows with label 'bs':", len(df_fake_bs),"\nNumber of rows with other labels:", len(df_fake_notbs))

Number of rows with label 'bs': 11492 
Number of rows with other labels: 1507


In [28]:
df_fake_bs_sample = df_fake_bs.sample(frac= 0.05)
df_fake_new = pd.concat([df_fake_bs_sample, df_fake_notbs])
df_fake_new = df_fake_new.sample(frac=1)
df_fake_new

,text,bias,bs,conspiracy,fake,hate,junksci,satire,state
6459,"Keywords: beauty , blackheads , blackheads tre...",False,False,False,False,False,True,False,False
4933,Wikileaks Email: Clinton Operative Thinks “Bla...,False,False,True,False,False,False,False,False
7269,"By Sean Colarossi on Mon, Oct 31st, 2016 at 8:...",True,False,False,False,False,False,False,False
300,By Nadia Prupis Canadian free speech advocates...,False,False,True,False,False,False,False,False
673,Juror explanation for Ammon Bundy verdict 11/0...,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
2342,Dr. David Duke and Prof. Kevin MacDonald on Du...,False,False,False,False,True,False,False,False
1614,"Email \nIf his campaign is any indication, Don...",False,False,False,False,False,False,True,False
321,By Catherine J. Frompovich This is the continu...,False,False,True,False,False,False,False,False
2237,Christie Lost Republicans The 2012 Election. H...,True,False,False,False,False,False,False,False


In [29]:
train, test = train_test_split(df_fake_new, test_size=0.2)

In [30]:
X2_train = train["text"]
X2_test = test["text"]

label_names = ["bias", "bs", "conspiracy", "fake", "hate", "junksci", "satire", "state"]
y2_train = train[label_names].values
y2_test = test[label_names].values

In [31]:
tokenizer2 = Tokenizer(num_words=MAX_VOCAB_SIZE,
                      lower=True,
                      char_level=False,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      split=' ')

tokenizer2.fit_on_texts(X2_train.tolist())

train_word_index_2 = tokenizer2.word_index
train_embedding_weights_2 = np.zeros((len(train_word_index_2)+1, EMBEDDING_DIM))

for word,index in train_word_index_2.items():
    if word in word2vec:
        train_embedding_weights_2[index,:] = word2vec[word]
    else:
        train_embedding_weights_2[index,:] = np.random.rand(EMBEDDING_DIM)

print("embedding dim:", train_embedding_weights_2.shape)

embedding dim: (44410, 300)


In [32]:
training_sequences_2 = tokenizer2.texts_to_sequences(X2_train.tolist())
train_cnn_data_2 = pad_sequences(training_sequences_2, maxlen=MAX_SEQUENCE_LENGTH-1)

test_sequences_2 = tokenizer2.texts_to_sequences(X2_test.tolist())
test_cnn_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH-1)

print(len(train_cnn_data_2))

1665


In [33]:
batch_size_2 = 256
labels_2 = 8
num_epochs_2 = 30

In [34]:
model2 = ConvNet(train_embedding_weights_2, MAX_SEQUENCE_LENGTH-1, len(train_word_index_2)+1, EMBEDDING_DIM, labels_2, False)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 199)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 199, 300)             1332300   ['input_2[0][0]']             
                                                          0                                       
                                                                                                  
 conv1d_4 (Conv1D)           (None, 197, 128)             115328    ['embedding_1[0][0]']         
                                                                                                  
 conv1d_5 (Conv1D)           (None, 196, 128)             153728    ['embedding_1[0][0]']   

In [35]:
tf.config.run_functions_eagerly(True)

In [ ]:
hist = model2.fit(train_cnn_data_2, y2_train, epochs=num_epochs_2, validation_split=0.1, shuffle=True, batch_size=batch_size_2)

In [48]:
y2_predicted = model2.predict(test_cnn_data_2, batch_size=1024, verbose=1)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 1s 802ms/step


In [49]:
df2_pred = pd.DataFrame(y2_predicted)
df2_pred = df2_pred.where(df2_pred!=0).rank(1, ascending=False, method='dense').eq(1).astype(int)
np2_pred = df2_pred.to_numpy()

In [50]:
accuracy_score(y2_test, np2_pred)

0.5371702637889688

In [51]:
pd.DataFrame(np2_pred).sample(frac= 0.1)

,0,1,2,3,4,5,6,7
317,0,1,0,0,0,0,0,0
137,0,0,1,0,0,0,0,0
261,0,1,0,0,0,0,0,0
139,1,0,0,0,0,0,0,0
181,0,0,1,0,0,0,0,0
385,0,1,0,0,0,0,0,0
283,1,0,0,0,0,0,0,0
119,0,1,0,0,0,0,0,0
188,0,1,0,0,0,0,0,0
31,0,0,0,0,0,0,0,1


# Check Models Compatibility
(using the model from Part 2 on the fake news obtained from Part 1)

In [52]:
df_combine = pd.DataFrame(X1_test)
df_combine["label"] = y1_pred
df_combine = df_combine.reset_index(drop=True)

df_combine_fake = df_combine.loc[df_combine['label'] == 0.0]

df_sample_fake = df_combine_fake.sample(frac=0.1)
df_sample_fake

,text,label
3407,Did you In some individuals born with the cong...,0.0
3747,The ministration Management System has started...,0.0
4887,if he solves all his cases in minutes or has d...,0.0
3292,J Trump made coal a centerpiece of his holding...,0.0
3788,Former National Guardsman turned State sympath...,0.0
...,...,...
3096,is AND fucked Time to airlift in cases of,0.0
2836,Will in,0.0
6553,die oder hat die um um die die Die die die Lan...,0.0
3635,has asserted that democracy can only be upheld...,0.0


In [53]:
combined_sample_tokenised = tokenizer2.texts_to_sequences(df_sample_fake["text"].tolist())
combined_sample_train_data = pad_sequences(combined_sample_tokenised, maxlen=MAX_SEQUENCE_LENGTH-1)

In [57]:
combined_predicted = model2.predict(combined_sample_train_data, batch_size=1024, verbose=1)

df_combined_pred = pd.DataFrame(combined_predicted)
df_combined_pred = df_combined_pred.where(df_combined_pred!=0).rank(1, ascending=False, method='dense').eq(1).astype(int)

df_combined_pred.columns = label_names 
df_combined_pred["text"] = df_sample_fake["text"].values
df_combined_pred


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 1s 615ms/step


,bias,bs,conspiracy,fake,hate,junksci,satire,state,text
0,1,0,0,0,0,0,0,0,Did you In some individuals born with the cong...
1,0,1,0,0,0,0,0,0,The ministration Management System has started...
2,1,0,0,0,0,0,0,0,if he solves all his cases in minutes or has d...
3,0,1,0,0,0,0,0,0,J Trump made coal a centerpiece of his holding...
4,1,0,0,0,0,0,0,0,Former National Guardsman turned State sympath...
...,...,...,...,...,...,...,...,...,...
377,1,0,0,0,0,0,0,0,is AND fucked Time to airlift in cases of
378,1,0,0,0,0,0,0,0,Will in
379,0,0,0,0,1,0,0,0,die oder hat die um um die die Die die die Lan...
380,0,1,0,0,0,0,0,0,has asserted that democracy can only be upheld...


# Test both models on input

In [19]:
text_news = input("Insert News")

In [20]:
text_tokenised_1 = tokenizer1.texts_to_sequences([[text_news]])
text_tokenised_1_data = pad_sequences(text_tokenised_1, maxlen=MAX_SEQUENCE_LENGTH-1)

In [66]:
text_1_predicted = model1.predict(text_tokenised_1_data, batch_size=1024, verbose=1)
text_1_predicted = np.round(np.array(text_1_predicted))
text_1_predicted = np.bool8(text_1_predicted)
text_1_predicted[0,0]


1/1 [==============================] - 0s 28ms/step


False

In [67]:
df_text2_pred = pd.DataFrame(None)

if text_1_predicted[0,0] == 0:
    text_tokenised_2 = tokenizer2.texts_to_sequences([[text_news]])
    text_tokenised_2_data = pad_sequences(text_tokenised_2, maxlen=MAX_SEQUENCE_LENGTH-1)
    text_2_predicted = model2.predict(text_tokenised_2_data, batch_size=1024, verbose=1)
    df_text2_pred = pd.DataFrame(text_2_predicted)
    df_text2_pred = df_text2_pred.where(df_text2_pred!=0).rank(1, ascending=False, method='dense').eq(1).astype(int)
    df_text2_pred.columns = label_names 
    
print(df_text2_pred)



1/1 [==============================] - 0s 28ms/step
   bias  bs  conspiracy  fake  hate  junksci  satire  state
0     1   0           0     0     0        0       0      0


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
